In [4]:
!pip install -r C:/MSBACKUP/Programovani/codingenterpreneurs/analytics-api/requirements.txt

  Using cached psycopg-3.2.9-py3-none-any.whl.metadata (4.5 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached psycopg_binary-3.2.9-cp312-cp312-win_amd64.whl.metadata (3.0 kB)
Using cached psycopg-3.2.9-py3-none-any.whl (202 kB)
Using cached psycopg_binary-3.2.9-cp312-cp312-win_amd64.whl (2.9 MB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)

   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ---------------------------------------- 0/3 [tzdata]
   ------------- -------------------------- 1/3 [psycopg-binary]
   -------------------------- ------------- 2/3 [psycopg]
   -------------------------- ------------- 2/3 [psycopg]
   -------------------------- ------------- 2/3 [psycopg]
   -------------------------- ------------- 2/3 [psycopg]
   ---------------------------------------- 3/3 [psycopg]



In [5]:
import sys
from pathlib import Path

In [ ]:
src_path = Path("../src").resolve()
sys.path.append(str(src_path))

WindowsPath('C:/MSBACKUP/Programovani/codingenterpreneurs/analytics-api/src')

In [11]:
from sqlmodel import Session, select
from api.events.models import EventModel
from api.db.session import engine

In [16]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.desc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds":True})
    print(compiled_query)
    print("")
    print(str(query))
    # results = session.exec(query).all()
    # print(results)

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at DESC
 LIMIT 10

SELECT eventmodel.id, eventmodel.time, eventmodel.page, eventmodel.description, eventmodel.updated_at 
FROM eventmodel ORDER BY eventmodel.updated_at DESC
 LIMIT :param_1


In [17]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.desc()).limit(10)
    results = session.exec(query).all()
    print(results)

[EventModel(page='/pages', updated_at=datetime.datetime(2025, 8, 9, 15, 44, 11, 842414, tzinfo=datetime.timezone.utc), id=10000, time=datetime.datetime(2025, 8, 9, 15, 44, 11, 842406, tzinfo=datetime.timezone.utc), description='my description'), EventModel(page='/pricing', updated_at=datetime.datetime(2025, 8, 9, 15, 44, 11, 705756, tzinfo=datetime.timezone.utc), id=9999, time=datetime.datetime(2025, 8, 9, 15, 44, 11, 705747, tzinfo=datetime.timezone.utc), description='my description'), EventModel(page='/pricing', updated_at=datetime.datetime(2025, 8, 9, 15, 44, 11, 566533, tzinfo=datetime.timezone.utc), id=9998, time=datetime.datetime(2025, 8, 9, 15, 44, 11, 566519, tzinfo=datetime.timezone.utc), description='my description'), EventModel(page='/pages', updated_at=datetime.datetime(2025, 8, 9, 15, 44, 11, 434285, tzinfo=datetime.timezone.utc), id=9997, time=datetime.datetime(2025, 8, 9, 15, 44, 11, 434275, tzinfo=datetime.timezone.utc), description='my description'), EventModel(page='/

In [ ]:
from timescaledb.hyperfunctions import time_bucket
from pprint import pprint
from sqlalchemy import func
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket = time_bucket("1 day", EventModel.time)
    pages = ['/about', '/contact', '/pages', '/pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) + timedelta(hours=1)
    query = (
        select(
            bucket,
            EventModel.page,
            func.count())
        .where(
            EventModel.time > start,
            EventModel.time <= finish,
            EventModel.page.in_(pages)
        )
        .group_by(
            bucket,
            EventModel.page
        )
        .order_by(
            bucket,
            EventModel.page
        )
    )
    compiled_query = query.compile(compile_kwargs={"literal_binds":True})
    pprint(compiled_query)
    results = session.exec(query).fetchall()
    pprint(results)

[(datetime.datetime(2025, 8, 11, 0, 0, tzinfo=datetime.timezone.utc), '/about', 243),
 (datetime.datetime(2025, 8, 11, 0, 0, tzinfo=datetime.timezone.utc), '/contact', 250),
 (datetime.datetime(2025, 8, 11, 0, 0, tzinfo=datetime.timezone.utc), '/pages', 245),
 (datetime.datetime(2025, 8, 11, 0, 0, tzinfo=datetime.timezone.utc), '/pricing', 262)]
